PROJECT

In [3]:
print(1)

1


In [1]:
# CELL: Full Pipeline - ADVANCED (GridSearchCV + Feature Engineering + SHAP)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier # Using RandomForest
import shap
import matplotlib.pyplot as plt
from IPython.display import display # Explicitly import display
import warnings
warnings.filterwarnings('ignore')

# --- 1. Load Data ---
print("Loading data...")
file_path = 'dataset.csv'

try:
    df = pd.read_csv(file_path)
    print(f"Successfully loaded '{file_path}'")
except FileNotFoundError:
    print(f"ERROR: Could not find the file '{file_path}'.")
    raise
except Exception as e:
    print(f"An error occurred loading the file: {e}")
    raise

# --- 2. Preprocessing ---
print("Preprocessing...")

# Drop index, encounter_id, and patient_id (not predictive)
df = df.drop(['index', 'encounter_id', 'patient_id'], axis=1)

# Handle weight (unchanged)
df['weight'] = df['weight'].replace('?', np.nan)
df['weight'] = df['weight'].str.replace('[', '').str.replace(')', '').str.split('-').str[0]
df['weight'] = pd.to_numeric(df['weight'], errors='coerce')

# Handle age (unchanged)
age_map = {
    '[0-10)': 5, '[10-20)': 15, '[20-30)': 25, '[30-40)': 35, '[40-50)': 45,
    '[50-60)': 55, '[60-70)': 65, '[70-80)': 75, '[80-90)': 85, '[90-100)': 95
}
df['age'] = df['age'].map(age_map)

# --- NEW: ADVANCED FEATURE ENGINEERING (Diagnosis Grouping) ---
print("Grouping diagnosis codes into categories...")
# We will group diag_1, diag_2, and diag_3
diag_cols = ['diag_1', 'diag_2', 'diag_3']

# First, convert codes to numeric (as before)
for col in diag_cols:
    df[col] = pd.to_numeric(df[col].astype(str).str.extract('(\d+)')[0], errors='coerce')

# Define the ICD-9 categories
def group_diagnosis(icd9_code):
    if pd.isna(icd9_code):
        return 'Missing'
    icd9_code = float(icd9_code)
    if (icd9_code >= 390 and icd9_code <= 459) or icd9_code == 785:
        return 'Circulatory'
    if (icd9_code >= 460 and icd9_code <= 519) or icd9_code == 786:
        return 'Respiratory'
    if (icd9_code >= 520 and icd9_code <= 579) or icd9_code == 787:
        return 'Digestive'
    if icd9_code == 250:
        return 'Diabetes'
    if icd9_code >= 800 and icd9_code <= 999:
        return 'Injury'
    if icd9_code >= 710 and icd9_code <= 739:
        return 'Musculoskeletal'
    if icd9_code >= 580 and icd9_code <= 629 or icd9_code == 788:
        return 'Genitourinary'
    if icd9_code >= 140 and icd9_code <= 239:
        return 'Neoplasms'
    return 'Other'

# Apply the grouping
for col in diag_cols:
    df[col] = df[col].apply(group_diagnosis)
    
# We now have categorical diagnosis codes instead of ~800 numbers.
# We also drop diag_4 and diag_5 as they are less important and for simplicity
df = df.drop(['diag_4', 'diag_5'], axis=1)
# --- END NEW FEATURE ENGINEERING ---

# Handle X1-X25 medications (unchanged)
med_cols = [f'X{i}' for i in range(1, 26)]
for col in ['X1', 'X2']:
    df[col] = df[col].replace('None', 0)
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
change_map = {'No': 0, 'Up': 1, 'Down': -1, 'Steady': 0}
for col in med_cols[2:]:
    df[col] = df[col].map(change_map).fillna(0)

# Categorical encoding
# Add our new diag_cols to the list
cat_cols = ['race', 'gender', 'medical_specialty', 'change', 'diabetesMed', 'diag_1', 'diag_2', 'diag_3']
for col in cat_cols:
    df[col] = df[col].fillna('missing')
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Fill remaining NaNs
df = df.fillna(df.median(numeric_only=True))

# Feature Engineering (from last time)
print("Creating total_visits and med_changes features...")
df['total_visits'] = df['number_inpatient'] + df['number_emergency'] + df['number_outpatient']
med_change_cols = [f'X{i}' for i in range(3, 26)] # X3 to X25
df['med_changes'] = (df[med_change_cols] != 0).sum(axis=1) # Count any 'Up' or 'Down'

# Target
y = df['readmitted']
X = df.drop('readmitted', axis=1)

# --- 3. Train-Test Split ---
print("Splitting data...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# --- 4. Hyperparameter Tuning with GridSearchCV ---
print("Starting Hyperparameter Tuning (this may take 5-10 minutes)...")

# Define the model we want to tune
rf = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)

# Define the grid of parameters to search
# NOTE: This is a small grid to save time. Research papers use much larger grids.
param_grid = {
    'n_estimators': [100, 200, 300], # Number of trees
    'max_depth': [10, 20, 30]         # Max depth of each tree
}

# Set up the GridSearch
# We are optimizing for 'f1_weighted' to match the research papers
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3, # 3-fold cross-validation
    scoring='f1_weighted', # This is the score we want to maximize
    n_jobs=-1, # Use all available cores
    verbose=2  # Prints progress
)

# Run the search
grid_search.fit(X_train, y_train)

# Get the best model
model = grid_search.best_estimator_

print("\n--- Model Evaluation ---")
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best f1_weighted score from tuning: {grid_search.best_score_:.4f}")

# Evaluate the final model on the test set
y_pred = model.predict(X_test)
print("\nFinal Test Set Results:")
print(classification_report(y_test, y_pred, target_names=['Not Readmitted (0)', 'Readmitted (1)']))


# --- 5. SHAP Explainability (Modern API) ---
# (This section is unchanged, but will now use the *tuned* model)
print("Computing SHAP values...")
shap.initjs()
explainer = shap.TreeExplainer(model)
explanation_object = explainer(X_test)


# --- 6. SHAP Plots (Separated for Readability) ---
# (This section is unchanged)
print("\n--- Generating SHAP Plots (One by One) ---")
explanation_class_1 = explanation_object[:, :, 1]

print("Displaying Interactive Force Plot (for first prediction, Class 1)...")
display(shap.force_plot(explanation_object[0, :, 1]))

print("\nGenerating Waterfall Plot...")
plt.figure()
shap.waterfall_plot(explanation_class_1[0], max_display=10, show=False)
plt.show()

print("\nGenerating Feature Importance (Bar Plot)...")
plt.figure()
shap.summary_plot(explanation_class_1, plot_type="bar", max_display=15, show=False)
plt.show()

print("\nGenerating Beeswarm Plot...")
plt.figure()
shap.summary_plot(explanation_class_1, max_display=15, show=False)
plt.show()

print("\nGenerating Dependence Plots...")
shap_values_class_1 = explanation_class_1.values
feature_names = explanation_class_1.feature_names
top_features_indices = np.argsort(np.abs(shap_values_class_1).mean(0))[-3:][::-1]

for i, idx in enumerate(top_features_indices):
    plt.figure()
    print(f"Plotting dependence for: {feature_names[idx]}")
    shap.dependence_plot(
        feature_names[idx], 
        shap_values_class_1, 
        X_test,
        show=False
    )
    plt.show()


# --- 7. Print Top 10 Important Features ---
# (This section is unchanged)
print("\nTop 10 Most Important Features (by mean |SHAP|):")
shap_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': np.abs(shap_values_class_1).mean(0)
}).sort_values('importance', ascending=False)
print(shap_importance.head(10).to_string(index=False))

print("\nAll SHAP plots generated successfully!")

c:\Users\utente\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Loading data...
Successfully loaded 'dataset.csv'
Preprocessing...
Grouping diagnosis codes into categories...
Creating total_visits and med_changes features...
Splitting data...
Starting Hyperparameter Tuning (this may take 5-10 minutes)...
Fitting 3 folds for each of 9 candidates, totalling 27 fits


KeyboardInterrupt: 